In [1]:
#include <memory>
#include <vector>

#include <iostream>

# **Duck Typing en C++**

<img width="384" height="384" src="pictures/duck-prints.svg">

Image by [svgrepo.com](https://www.svgrepo.com/svg/32836/duck-prints)

### Alberto Lorenzo Márquez

* Ingeniero aeroespacial

* Desarrollador de software (**C++**, **Fortran**, **Python**, **Javascript**)

* **Github**: https://github.com/newlawrence

* **Twitter**: [@newlawrence](https://twitter.com/newlawrence)

# Polimorfismo

## Polimorfismo dinámico

El tradicional basado en **herencia** y las **funciones virtuales**.

In [2]:
class IFoo {
public:
    virtual void bar() const = 0;
    virtual ~IFoo() = default;
};

In [3]:
class Foo1 final : public IFoo {
public:
    void bar() const override {
        std::cout << "I'm a Foo1 instance" << std::endl;
    }
};

class Foo2 final : public IFoo {
public:
    void bar() const override {
        std::cout << "I'm a Foo2 instance" << std::endl;
    }
};

In [4]:
std::vector<std::unique_ptr<IFoo>> foo_ptrs;
foo_ptrs.emplace_back(std::make_unique<Foo1>(Foo1()));
foo_ptrs.emplace_back(std::make_unique<Foo2>(Foo2()));

for (auto const& foo : foo_ptrs)
    foo->bar();

I'm a Foo1 instance
I'm a Foo2 instance


¿Utilizar una clase que no pertenece a la jerarquía?

In [5]:
class Foo3 {
public:
    void bar() const { std::cout << "I'm a Foo3 instance" << std::endl; }
};

### Principales inconvenientes

* Semántica por referencia.

* Requiere de código adicional para adaptar código de terceros.

## Polimorfismo estático

Las plantillas de toda la vida.

In [6]:
template<typename... Args>
void call_bar(Args&&... foos) { (foos.bar(), ...); }

In [7]:
call_bar(Foo1(), Foo2(), Foo3());

I'm a Foo1 instance
I'm a Foo2 instance
I'm a Foo3 instance


¿Almacenar distantas instancias de clases de plantilla en un contenedor...

### Tupla

... dinámico en tiempo de ejecución...

### Variant

... para cualquier tipo que cumpla la interfaz?

### ¿Any?

### Principal inconveniente

* Comunicación entre distintas unidades de traducción sólo para tipos conocidos.

### Repasando...

...la herencia permite utilizar tipos desconocidos a través de las interfaces.

...las plantillas permiten generar código de forma automática.

## Combinando polimorfismos

In [8]:
class FooConcept {
public:
    virtual void bar() const = 0;
    virtual ~FooConcept() = default;
};

Utilizando la herencia proporcionando una clase base para las plantillas, se consigue que estas no tengan por qué ser conocidas entre distintas unidades de traducción.

In [9]:
template<typename T>
class FooModel final : public FooConcept {
    T self_;

public:
    template<typename U>
    FooModel(U&& self) : self_{ std::forward<U>(self) } {}

    void bar() const override { self_.bar(); }
};

Utilizando las plantillas, se consigue liberar al usuario de tener que escribir código para adaptarse a la interfaz.

## Borrado de tipos

In [10]:
class Foo {
    std::unique_ptr<FooConcept> self_;

public:
    template<typename T>
    Foo(T&& self)
        : self_{ std::make_unique<FooModel<T>>(std::forward<T>(self)) }
    {}

    void bar() const { self_->bar(); }
};

Finalmente, ocultando las dos técnicas anteriores bajo un mismo tipo, se consigue proporcionar semántica por valor.

In [11]:
std::vector<Foo> foos;
foos.emplace_back(Foo1());  // Same hierarchy
foos.emplace_back(Foo2());  // Same hierarchy
foos.emplace_back(Foo3());  // Foreign class

for (auto const& foo : foos)
    foo.bar();

I'm a Foo1 instance
I'm a Foo2 instance
I'm a Foo3 instance


### Duck Typing

> When I see a bird that walks like a duck and swims like a duck and quacks like a duck, I call that bird a duck.
>
> <cite><b>James Whitcomb Riley</b></cite>